# A* Algorithm Implmentation: Closest Store by BART

**Import Statements**

In [14]:
import neo4j
import math
import numpy as np
import pandas as pd
import re
import random
from sklearn.neighbors import BallTree


import psycopg2
from scipy import spatial
from geographiclib.geodesic import Geodesic

**Driver Connection**

In [15]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)
cursor = connection.cursor()
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

**Helper Functions**

In [16]:
## Pandas Functions

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [17]:
## Generic Neo4j Functions

def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query) 
    
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [18]:
## Create Nodes

def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    
def my_neo4j_create_customer_node(customer_id):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Customer {
                        customer_id: $customer_id
                    })
    
    """
    
    session.run(query, customer_id=customer_id)
    
def my_neo4j_create_locker_node(locker_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Locker {
                        locker_name: $locker_name
                    })
    
    """
    
    session.run(query, locker_name=locker_name)

In [19]:
## Create One-Way Relationships 

def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

def create_relationship_one_way_customer_station(from_customer, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Customer), 
          (to:Station)
    WHERE from.customer_id = $from_customer and to.name = $to_station
    CREATE (from)-[:DIST {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_customer=from_customer, to_station=to_station, weight=weight)
    
def create_relationship_one_way_locker_station(from_station, locker_name, weight):
    """
    create a relationship one way between two stations with a weight
    note: may need to add locker station
    """
    
    query = """
    
    MATCH (from:Station), 
          (to:Locker)
    WHERE from.name = $from_station and to.locker_name = $locker_name 
    CREATE (from)-[:DIST {weight: $weight}]->(to)
    
    """
    session.run(query, from_station=from_station, locker_name=locker_name, weight=weight)


In [20]:
# Create Two-Way Relationships 
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)



    
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)




In [21]:
## Distance/Random Latitude Functions 

def generate_random_lat_lon(center_lat, center_lon, radius_miles):
    """
    Generate a random latitude and longitude pair within a given radius of a center point.

    Parameters:
    - center_lat (float): Latitude of the center point.
    - center_lon (float): Longitude of the center point.
    - radius_miles (float): Radius in miles.

    Returns:
    - Tuple: Random latitude and longitude pair.
    """
    # Convert latitude and longitude from degrees to radians
    center_lat_rad = math.radians(center_lat)
    center_lon_rad = math.radians(center_lon)

    # Generate a random angle and distance within the specified radius
    random_angle = random.uniform(0, 2 * math.pi)
    random_distance = random.uniform(0, radius_miles)

    # Calculate the new latitude and longitude
    new_lat_rad = center_lat_rad + (random_distance / 3959) * math.cos(random_angle)
    new_lon_rad = center_lon_rad + (random_distance / 3959) * math.sin(random_angle)

    # Convert back to degrees
    new_lat = math.degrees(new_lat_rad)
    new_lon = math.degrees(new_lon_rad)

    return new_lat, new_lon

In [22]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

## Graph Algorithms
def my_neo4j_shortest_path_to_locker(from_customer, to_locker):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', ['Customer', 'Locker', 'Station'], ['LINK', 'DIST'], {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Customer {customer_id: $source}), (target:Locker {locker_name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).customer_id AS from,
        gds.util.asNode(targetNode).locker_name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_customer, target=to_locker)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        nodes[0] = 'from customer_id ' + str(from_customer)
        nodes[-1] = to_locker
        
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
        
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1

## Graph Creation Process 
1. Getting appropriate data 
    a. query customer location data from the AGM database, only including customers whose zip code is included in a county where BART runs 
    b. Randomly generate latitude/longitude coordinates for each customer, using a 1 mile radius from the center of their corresponding zip code 
    c. Identify the closest station to each customer based on the random coordinates

### Customer Zip Code -> Random (lat, long) pairs
- note: filtered customers based on zip codes in counties where BART operates 

In [23]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT customers.customer_id, customers.zip, zip_codes.latitude, zip_codes.longitude
FROM customers
left join zip_codes 
on customers.zip = zip_codes.zip 

"""
bart_zip_codes = [
    # San Francisco County
    "94102", "94103", "94104", "94105", "94107", "94108", "94109", "94110", "94111", "94112", "94114", "94115", "94116", "94117", "94118", "94121", "94122", "94123", "94124", "94127", "94129", "94130", "94131", "94132", "94133", "94134",
    
    # San Mateo County
    "94401", "94402", "94403", "94404",
    "94061", "94062", "94063", "94065",
    # Palo Alto (part of)
    "94306",
    
    # Santa Clara County
    "95110", "95111", "95112", "95116", "95117", "95118", "95122", "95123", "95124", "95125", "95126", "95127", "95128", "95129", "95130", "95131", "95132", "95133", "95134", "95135", "95136", "95138", "95139", "95148",
    "94301", "94303", "94304", "94306",
    "94040", "94041", "94043",
    "94085", "94086", "94087", "94089",
    
    # Alameda County
    "94601", "94602", "94603", "94605", "94606", "94607", "94608", "94609", "94610", "94611", "94612", "94618", "94619", "94621",
    "94702", "94703", "94704", "94705", "94707", "94708", "94709", "94710", "94720",
    "94536", "94538", "94539", "94555",
    "94541", "94542", "94544", "94545",
    "94577", "94578", "94579",
    
    # Contra Costa County
    "94595", "94596", "94597", "94598",
    "94518", "94519", "94520", "94521",
    "94801", "94804", "94805", "94806"
]

# Displaying the list

customers = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
customers = customers[customers['zip'].isin(bart_zip_codes)]
customers.head()

,customer_id,zip,latitude,longitude
0,1,94609,37.8343,-122.2643
1,2,94609,37.8343,-122.2643
2,3,94609,37.8343,-122.2643
3,4,94609,37.8343,-122.2643
4,5,94609,37.8343,-122.2643


In [24]:
random_locs = customers.apply(lambda x: generate_random_lat_lon(x['latitude'], x['longitude'], 1), axis=1)
customers["random_lat"] = random_locs.apply(lambda x: x[0])
customers["random_long"] = random_locs.apply(lambda x: x[1])
rand_customers_loc = customers[['customer_id', 'zip', 'random_lat', 'random_long']]
rand_customers_loc

,customer_id,zip,random_lat,random_long
0,1,94609,37.842808,-122.267196
1,2,94609,37.838548,-122.263928
2,3,94609,37.834342,-122.266797
3,4,94609,37.824752,-122.264462
4,5,94609,37.836569,-122.272701
...,...,...,...,...
8127,8123,94536,37.564414,-121.983674
8128,8124,94536,37.572449,-121.986493
8129,8125,94536,37.569559,-121.987779
8130,8126,94536,37.573543,-121.986847


In [25]:
distinct_pairs_count_rand = rand_customers_loc[['random_lat', 'random_long']].apply(tuple, axis=0).nunique()
distinct_pairs_count_og = customers[['latitude', 'longitude']].apply(tuple, axis=0).nunique()
distinct_pairs_count_og, distinct_pairs_count_rand

(latitude     76
 longitude    76
 dtype: int64,
 random_lat     5673
 random_long    5673
 dtype: int64)

## Get Stations and Coordinates

In [26]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT *
FROM stations

"""

stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations.head()

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0


## Building customers nodes, and their relations to their closest stations

In [27]:
rand_customers_loc['random_lat_RAD'] = np.deg2rad(rand_customers_loc['random_lat'].values)
rand_customers_loc['random_long_RAD'] = np.deg2rad(rand_customers_loc['random_long'].values)
    
bt = BallTree(np.deg2rad(stations[['latitude', 'longitude']].values), metric='haversine')
dists, inds = bt.query(rand_customers_loc[['random_lat_RAD', 'random_long_RAD']], k = 1)

#convert distances to miles and flatten multi-level indices 
dists = dists.flatten()
inds = inds.flatten()

dists = dists * 3958.8 

/tmp/ipykernel_164/3007730496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rand_customers_loc['random_lat_RAD'] = np.deg2rad(rand_customers_loc['random_lat'].values)


In [28]:
rand_customers_loc['distance_miles'] = dists
rand_customers_loc['travel_time_secs'] = (dists / 2.5) * 3600

rand_customers_loc['closest_station'] = stations.loc[inds,'station'].values
rand_customers_loc

,customer_id,zip,random_lat,random_long,random_lat_RAD,random_long_RAD,distance_miles,travel_time_secs,closest_station
0,1,94609,37.842808,-122.267196,0.660482,-2.133965,0.724703,1043.571919,Ashby
1,2,94609,37.838548,-122.263928,0.660407,-2.133908,0.733926,1056.853379,MacArthur
2,3,94609,37.834342,-122.266797,0.660334,-2.133958,0.421019,606.267697,MacArthur
3,4,94609,37.824752,-122.264462,0.660166,-2.133917,0.286878,413.104960,MacArthur
4,5,94609,37.836569,-122.272701,0.660373,-2.134061,0.645991,930.226514,MacArthur
...,...,...,...,...,...,...,...,...,...
8127,8123,94536,37.564414,-121.983674,0.655623,-2.129017,0.615000,885.599765,Fremont
8128,8124,94536,37.572449,-121.986493,0.655763,-2.129066,1.166531,1679.804726,Fremont
8129,8125,94536,37.569559,-121.987779,0.655713,-2.129088,1.033942,1488.876515,Fremont
8130,8126,94536,37.573543,-121.986847,0.655782,-2.129072,1.242605,1789.351295,Fremont


In [29]:
print("Number of Customers Closest to Station")
rand_customers_loc.loc[:, 'closest_station'].value_counts()

Number of Customers Closest to Station


Fruitvale               506
Rockridge               428
Downtown Berkeley       419
MacArthur               323
Civic Center            309
North Berkeley          297
Richmond                280
16th Street Mission     239
West Oakland            206
19th Street             206
Lake Merritt            196
El Cerrito del Norte    178
El Cerrito Plaza        178
Ashby                   175
Walnut Creek            173
24th Street Mission     172
Glen Park               154
San Leandro             143
Coliseum                142
Embarcadero             141
Bay Fair                128
Balboa Park             121
Powell Street           108
Montgomery Street        91
Pleasant Hill            62
Daly City                45
Concord                  44
Millbrae                 38
Orinda                   35
12th Street              32
North Concord            29
South Hayward            25
OAK                      17
Union City               13
Hayward                   9
Fremont             

## Rebuilding the BART Graph 

In [30]:
## Starting Graph
## Wiping out the database 
my_neo4j_wipe_out_database()
## Nodes Post Wipeout
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [31]:
## Adding Station Departure and Arrival Nodes 
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)

## Verify the number of nodes 
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [32]:
query = """

SELECT station, line
FROM lines
ORDER BY station, line

"""

cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    station = row[0]
    line = row[1]
    node = line + ' ' + station
    my_neo4j_create_node(node)
    my_neo4j_create_relationship_one_way('depart ' + station, node, 0)
    my_neo4j_create_relationship_one_way(node, 'arrive ' + station, 0)
    
## Verify the number of nodes 
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [33]:
query = """

WITH line_to_line AS (
    SELECT l1.station, l1.line AS from_line, l2.line AS to_line
    FROM lines l1 JOIN lines l2 ON l1.station = l2.station 
    WHERE l1.line != l2.line)
    
SELECT stations.station, from_line, to_line, transfer_time
FROM line_to_line JOIN stations ON line_to_line.station = stations.station

"""

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    my_neo4j_create_relationship_one_way(from_line + ' ' + station, to_line + ' ' + station, transfer_time) 
    
my_neo4j_number_nodes_relationships()


-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [34]:
query = """

WITH line_to_line AS(
    SELECT l1.line, l1.station AS "from station", l2.station AS "to station"
    FROM lines l1 JOIN lines l2 ON l1.line = l2.line
    WHERE l1.station != l2.station AND (l1.sequence + 1) = (l2.sequence)
)

SELECT line, "from station", "to station", travel_time AS "travel time in seconds"
FROM line_to_line, travel_times
WHERE ("from station" = station_1 AND "to station" = station_2) OR ("from station" = station_2 AND "to station" = station_1 )
ORDER BY line, "from station", "to station"

"""

cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = int(row[3])
    my_neo4j_create_relationship_two_way(line + ' ' + from_station, line + ' ' + to_station, travel_time)
    
my_neo4j_number_nodes_relationships()


-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


## Testing Shortest Path on Rebuilt Graph 

### Creating Customer -> Departure Station Relationships 

In [35]:
rand_customers_loc.head()

,customer_id,zip,random_lat,random_long,random_lat_RAD,random_long_RAD,distance_miles,travel_time_secs,closest_station
0,1,94609,37.842808,-122.267196,0.660482,-2.133965,0.724703,1043.571919,Ashby
1,2,94609,37.838548,-122.263928,0.660407,-2.133908,0.733926,1056.853379,MacArthur
2,3,94609,37.834342,-122.266797,0.660334,-2.133958,0.421019,606.267697,MacArthur
3,4,94609,37.824752,-122.264462,0.660166,-2.133917,0.286878,413.104960,MacArthur
4,5,94609,37.836569,-122.272701,0.660373,-2.134061,0.645991,930.226514,MacArthur


**Creating Customer Nodes**

In [36]:
## Creating Customer Nodes 
connection.rollback()
my_neo4j_number_nodes_relationships()
cust_ids = rand_customers_loc['customer_id'].values
print(214 + len(cust_ids))
for i in range(len(cust_ids)):
    my_neo4j_create_customer_node(cust_ids[i])
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------
5887
-------------------------
  Nodes: 5887
  Relationships: 652
-------------------------


**Creating Customer -> Departure Station relationships** 
- note: travel time in seconds was used for the weights

In [37]:
connection.rollback()

for i in range(len(dists)):
    
    customer_node = cust_ids[i]
    departure_station = 'depart ' + rand_customers_loc['closest_station'].values[i]
    travel_time_weight = rand_customers_loc['travel_time_secs'].values[i]
    
    create_relationship_one_way_customer_station(customer_node, departure_station, travel_time_weight)

In [38]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 5887
  Relationships: 6325
-------------------------


### Creating Departure Station -> Pickup Locker Relationships 

In [39]:
stations.head()

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0


In [40]:
locker_stations = ['12th Street',
                 '16th Street Mission',
                 '19th Street',
                 '24th Street Mission',
                 'Bay Fair',
                 'Civic Center',
                 'Coliseum',
                 'Embarcadero',
                 'Fruitvale',
                 'Lake Merritt',
                 'MacArthur',
                 'Montgomery Street',
                 'Powell Street',
                 'San Leandro',
                 'West Oakland']

locker_df = stations[stations["station"].isin(locker_stations)]
locker_df

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
7,Bay Fair,37.697000,-122.126500,63
10,Civic Center,37.779861,-122.413498,325
11,Coliseum,37.753611,-122.196944,54
19,Embarcadero,37.793056,-122.397222,304
21,Fruitvale,37.774800,-122.224100,279
25,Lake Merritt,37.797773,-122.266588,309


In [41]:
## Creating Locker Nodes 
connection.rollback()
my_neo4j_number_nodes_relationships()

for i in range(locker_df.shape[0]):
    locker_name = locker_df['station'].values[i] + " AGM Pickup Locker"
    my_neo4j_create_locker_node(locker_name)
my_neo4j_number_nodes_relationships()


-------------------------
  Nodes: 5887
  Relationships: 6325
-------------------------
-------------------------
  Nodes: 5902
  Relationships: 6325
-------------------------


**Creating Arrival Station -> Pickup Locker relationships** 
- note: travel time = 120 seconds assuming non-zero time to account for pickup time

In [42]:
connection.rollback()
my_neo4j_number_nodes_relationships()

for i in range(locker_df.shape[0]):
    create_relationship_one_way_locker_station("arrive " + locker_df['station'].values[i], locker_df['station'].values[i] + " AGM Pickup Locker", 120)
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 5902
  Relationships: 6325
-------------------------
-------------------------
  Nodes: 5902
  Relationships: 6340
-------------------------


In [46]:
rand_customers_loc.head()

,customer_id,zip,random_lat,random_long,random_lat_RAD,random_long_RAD,distance_miles,travel_time_secs,closest_station
0,1,94609,37.842808,-122.267196,0.660482,-2.133965,0.724703,1043.571919,Ashby
1,2,94609,37.838548,-122.263928,0.660407,-2.133908,0.733926,1056.853379,MacArthur
2,3,94609,37.834342,-122.266797,0.660334,-2.133958,0.421019,606.267697,MacArthur
3,4,94609,37.824752,-122.264462,0.660166,-2.133917,0.286878,413.104960,MacArthur
4,5,94609,37.836569,-122.272701,0.660373,-2.134061,0.645991,930.226514,MacArthur


In [47]:
customer_id = 3
pickup_locker = "Embarcadero AGM Pickup Locker"

"""
['12th Street',
 '16th Street Mission',
 '19th Street',
 '24th Street Mission',
 'Bay Fair',
 'Civic Center',
 'Coliseum',
 'Embarcadero',
 'Fruitvale',
 'Lake Merritt',
 'MacArthur',
 'Montgomery Street',
 'Powell Street',
 'San Leandro',
 'West Oakland']
"""

my_neo4j_shortest_path_to_locker(customer_id, pickup_locker)


--------------------------------
   Total Cost:  1746
   Minutes:  29.1
--------------------------------
from customer_id 3, 0, 0
depart MacArthur, 606, 606
red MacArthur, 0, 606
red 19th Street, 180, 786
red 12th Street, 120, 906
red West Oakland, 300, 1206
red Embarcadero, 420, 1626
arrive Embarcadero, 0, 1626
Embarcadero AGM Pickup Locker, 120, 1746
